In [2]:
import pandas as pd
import numpy as np

In [3]:
df =pd.read_csv('../MachineHackElectronicPrice/train_FULL.csv')

In [4]:
df.head()

,Brand,Locality,City,State,Price,state_mean,state_median,state_sum,city_mean,city_median,...,758,759,760,761,762,763,764,765,766,767
0,1,878,8,2,15000,25613.621212,18000.0,6761996,25613.621212,18000.0,...,-0.019527,-0.561037,-0.666514,-0.295823,0.616575,-0.538386,0.190308,-0.254338,0.336432,0.615816
1,1,1081,4,0,18800,25613.621212,18000.0,6761996,25613.621212,18000.0,...,-0.531523,-1.012917,-0.641703,-0.121814,0.592504,-0.616559,0.271992,0.235450,-0.375156,0.019189
2,1,495,11,4,50000,25613.621212,18000.0,6761996,25613.621212,18000.0,...,-0.469713,-0.895722,-0.664055,-0.302316,0.349633,-0.544164,-0.312630,-0.328001,-0.004595,-0.044251
3,1,287,10,7,16500,25613.621212,18000.0,6761996,25613.621212,18000.0,...,-0.581632,-0.294631,-0.361210,0.074732,0.220956,-0.816656,-0.149960,-0.492918,0.081806,0.115384
4,1,342,4,0,26499,25613.621212,18000.0,6761996,25613.621212,18000.0,...,-0.359678,-0.931029,-0.978271,0.088294,0.333377,-0.654428,-0.352469,-0.162105,-0.088563,0.308029


In [14]:
def featureselection(train,y_train):
    from sklearn.linear_model import LinearRegression
    from sklearn.ensemble import ExtraTreesRegressor
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.feature_selection import SelectFromModel
    from lightgbm import LGBMRegressor
    from sklearn.feature_selection import chi2
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.feature_selection import RFE
    from sklearn.feature_selection import SelectKBest

    feature_name = train.columns.tolist()

    
    rf = RandomForestRegressor(n_estimators=100, random_state=7 )
    model_rf = SelectFromModel(rf ,threshold ='1.25*median')
    X_new_rf = model_rf.fit(train,y_train)
    embeded_rf_support = model_rf.get_support()
    embeded_rf_feature = train.loc[:,embeded_rf_support].columns.tolist()
    
    
    lgbc=LGBMRegressor(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)
    model_lgbc = SelectFromModel(lgbc,threshold = '1.25*median')
    X_new_lgbc = model_lgbc.fit(train,y_train)
    lgbc_rf_support = model_lgbc.get_support()
    lgbc_rf_feature = train.loc[:,lgbc_rf_support].columns.tolist()
    
    
    cor_support, cor_feature = cor_selector(train, y_train)
    
    
    X_norm = MinMaxScaler().fit_transform(train)
    chi_selector = SelectKBest(chi2, k=800)
    chi_selector.fit(X_norm, y_train)
    chi_support = chi_selector.get_support()
    chi_feature = train.loc[:,chi_support].columns.tolist()
    
    
    rfe_selector = RFE(estimator=LinearRegression(), n_features_to_select=823, step=10, verbose=5)
    rfe_selector.fit(X_norm, y_train)
    rfe_support = rfe_selector.get_support()
    rfe_feature = train.loc[:,rfe_support].columns.tolist()
    
    
    embeded_lr_selector = SelectFromModel(LinearRegression(), '1.25*median')
    embeded_lr_selector.fit(X_norm, y_train)

    embeded_lr_support = embeded_lr_selector.get_support()
    embeded_lr_feature = train.loc[:,embeded_lr_support].columns.tolist()
    
    
    pd.set_option('display.max_rows', None)
    
    feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                        'Random Forest':embeded_rf_support, 'LightGBM':lgbc_rf_support})
    
    feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
    feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
    feature_selection_df.index = range(1, len(feature_selection_df)+1)
    return feature_selection_df


    
    
    
def cor_selector(X, y):
    cor_list = []
    feature_name = X.columns.tolist()

    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-800:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
    
    


    

In [15]:
train = df.drop('Price',axis=1)

In [16]:
train.head()

,Brand,Locality,City,State,state_mean,state_median,state_sum,city_mean,city_median,city_sum,...,758,759,760,761,762,763,764,765,766,767
0,1,878,8,2,25613.621212,18000.0,6761996,25613.621212,18000.0,6761996,...,-0.019527,-0.561037,-0.666514,-0.295823,0.616575,-0.538386,0.190308,-0.254338,0.336432,0.615816
1,1,1081,4,0,25613.621212,18000.0,6761996,25613.621212,18000.0,6761996,...,-0.531523,-1.012917,-0.641703,-0.121814,0.592504,-0.616559,0.271992,0.235450,-0.375156,0.019189
2,1,495,11,4,25613.621212,18000.0,6761996,25613.621212,18000.0,6761996,...,-0.469713,-0.895722,-0.664055,-0.302316,0.349633,-0.544164,-0.312630,-0.328001,-0.004595,-0.044251
3,1,287,10,7,25613.621212,18000.0,6761996,25613.621212,18000.0,6761996,...,-0.581632,-0.294631,-0.361210,0.074732,0.220956,-0.816656,-0.149960,-0.492918,0.081806,0.115384
4,1,342,4,0,25613.621212,18000.0,6761996,25613.621212,18000.0,6761996,...,-0.359678,-0.931029,-0.978271,0.088294,0.333377,-0.654428,-0.352469,-0.162105,-0.088563,0.308029


In [17]:
y = df.Price

In [18]:
f = featureselection(train,y)

In [19]:
f

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
1,93,True,True,True,True,True,True,6
2,8,True,True,True,True,True,True,6
3,659,True,True,True,True,True,True,6
4,6,True,True,True,True,True,True,6
5,534,True,True,True,True,True,True,6
6,418,True,True,True,True,True,True,6
7,371,True,True,True,True,True,True,6
8,351,True,True,True,True,True,True,6
9,2,True,True,True,True,True,True,6
10,14,True,True,True,True,True,True,6
